In [68]:
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)

import os
import wget
from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline

In [69]:
data = pd.read_csv("data.csv")

In [70]:
data.head()

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,4,0,12/7/2009,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020


In [71]:
data["TARGET"] = data["IsBadBuy"]
data = data.drop(["IsBadBuy"], axis=1)

In [72]:
categorical_columns = ["Auction", "Make", "Model", "Trim", "SubModel", "Color",
                      "Transmission", "WheelTypeID", "Nationality", "Size", "TopThreeAmericanName",
                      "BYRNO", "VNZIP1", "VNST"]
useless_columns = ["WheelType", "PRIMEUNIT", "AUCGUART", "RefId"]

In [73]:
data = data.drop(useless_columns, axis=1)

In [74]:
data = data.dropna()

In [75]:
data["TARGET"] = LabelEncoder().fit_transform(data["TARGET"])

In [76]:
features = [col for col in categorical_columns] 
for feat in features:
    data[feat] = LabelEncoder().fit_transform(data[feat])

In [77]:
import time
import datetime
data["PurchDate"] = list(map(lambda x: int(time.mktime(datetime.datetime.strptime(x, "%m/%d/%Y").timetuple())), data["PurchDate"]))

In [78]:
data["PurchDate"] -= min(data["PurchDate"])
data["PurchDate"] //= 60 * 60 * 24

In [79]:
data.head()

,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,Color,Transmission,...,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost,TARGET
0,336,0,2006,3,16,533,131,213,12,0,...,8552.0,11597.0,12409.0,56,47,5,7100.0,0,1113,0
1,336,0,2004,5,5,0,92,730,14,0,...,9222.0,11374.0,12791.0,46,47,5,7600.0,0,1053,0
2,336,0,2005,4,5,795,97,284,7,0,...,5557.0,7146.0,8702.0,46,47,5,4900.0,0,1389,0
3,336,0,2004,5,5,591,97,147,13,0,...,2646.0,4375.0,5518.0,46,47,5,4100.0,0,630,0
4,336,0,2005,4,6,338,125,52,13,1,...,4384.0,6739.0,7911.0,46,47,5,4000.0,0,1020,0


In [80]:
data.to_csv("train.csv", index=False)